In [4]:
import numpy as np
import pandas as pd

%run BSE.py

BSE: Run completed.


In [5]:
def run_single_mm_experiment(
    sess_prefix,
    run_id,
    mm_type,
    mm_params,
    market_type="synthetic",
    end_time=60*10,
    seed=None
):
   
    import numpy as np
    import pandas as pd

    if seed is not None:
        np.random.seed(seed)

    trader_spec = {
        'buyers': [
            ('ZIC', 10)
        ],
        'sellers': [
            ('ZIC', 10)
        ],
        'market_makers': [
            (mm_type, 1, mm_params)
        ]
    }



    if market_type == "synthetic":

        supply = [
            {
                'from': 0,
                'to': end_time,
                'ranges': [(100, 200)],
                'stepmode': 'fixed'
            }
        ]

        demand = [
            {
                'from': 0,
                'to': end_time,
                'ranges': [(100, 200)],
                'stepmode': 'fixed'
            }
        ]

        order_sched = {
            'sup': supply,
            'dem': demand,
            'timemode': 'periodic',
            'interval': 1
        }


    elif market_type == "ibm_offset":

        offset_fn = offset_from_file(
            price_offset_filename="offset-ibm-1m-170831.csv",
            time_col=1,
            price_col=5,
            scale_factor=1,
            end_time=end_time
        )

        supply = [
            {
                'from': 0,
                'to': end_time,
                'ranges': [(100, 200)],
                'stepmode': 'fixed',
                'offsetfn': offset_fn
            }
        ]
        demand = [
            {
                'from': 0,
                'to': end_time,
                'ranges': [(100, 200)],
                'stepmode': 'fixed',
                'offsetfn': offset_fn
            }
        ]
        order_sched = {
                'sup': supply, 
                'dem': demand,
                'timemode': 'periodic',
                'interval': 1
        }

    else:
        raise ValueError("Unknown market type")

    # ----- run market session -----
    session_id = f"{sess_prefix}_{run_id}"

    dump_flags = {
        'dump_blotters': False,
        'dump_lobs': False,
        'dump_tape': False,
        'dump_strats': False,
        'dump_order_books': False,
        'dump_trader_stats': True,
        'dump_avgbals':True
    }

    market_session(
        sess_id=session_id,
        starttime=0,
        endtime=end_time,
        trader_spec=trader_spec,
        order_schedule=order_sched,
        dumpfile_flags=dump_flags,
        sess_vrbs=False
    )


    statsfile = f"{session_id}_avg_balance.csv"
    df = pd.read_csv(statsfile)
    if df.shape[0] == 0:
        profit = 0.0
    else:
        initial_balance = df.iloc[0, 1]
        final_balance = df.iloc[-1, 1]
        profit = final_balance - initial_balance

    return profit


In [6]:
MMM01_star_params = {
    "n_past_trades": 10,
    "bid_percent": 0.9,
    "ask_delta": 5
}

MMM02_params = MMM01_star_params.copy()

In [7]:
p1 = run_single_mm_experiment(
    sess_prefix="q4_sanity",
    run_id=0,
    mm_type="MMM01",
    mm_params=MMM01_star_params,
    market_type="synthetic",
    end_time=60*10
)

p2 = run_single_mm_experiment(
    sess_prefix="q4_sanity",
    run_id=1,
    mm_type="MMM02",
    mm_params=MMM02_params,
    market_type="synthetic",
    end_time=60*10
)

print("MMM01* profit:", p1)
print("MMM02  profit:", p2)

MMM01* profit: 431
MMM02  profit: 458


In [8]:
N_RUNS = 20
END_TIME = 60 * 10

results = []

for market in ["synthetic", "ibm_offset"]:
    for mm_type, params in [
        ("MMM01", MMM01_star_params),
        ("MMM02", MMM02_params)
    ]:
        for run in range(N_RUNS):
            print(f"Running {market}, {mm_type}, run {run}")
            profit = run_single_mm_experiment(
                sess_prefix="q4",
                run_id=f"{market}_{mm_type}_{run}",
                mm_type=mm_type,
                mm_params=params,
                market_type=market,
                end_time=END_TIME,
                seed=run
            )
            results.append({
                "market": market,
                "mm_type": mm_type,
                "run": run,
                "profit": profit
            })

df_q4 = pd.DataFrame(results)

df_q4.head()

Running synthetic, MMM01, run 0
Running synthetic, MMM01, run 1
Running synthetic, MMM01, run 2
Running synthetic, MMM01, run 3
Running synthetic, MMM01, run 4
Running synthetic, MMM01, run 5
Running synthetic, MMM01, run 6
Running synthetic, MMM01, run 7
Running synthetic, MMM01, run 8
Running synthetic, MMM01, run 9
Running synthetic, MMM01, run 10
Running synthetic, MMM01, run 11
Running synthetic, MMM01, run 12
Running synthetic, MMM01, run 13
Running synthetic, MMM01, run 14
Running synthetic, MMM01, run 15
Running synthetic, MMM01, run 16
Running synthetic, MMM01, run 17
Running synthetic, MMM01, run 18
Running synthetic, MMM01, run 19
Running synthetic, MMM02, run 0
Running synthetic, MMM02, run 1
Running synthetic, MMM02, run 2
Running synthetic, MMM02, run 3
Running synthetic, MMM02, run 4
Running synthetic, MMM02, run 5
Running synthetic, MMM02, run 6
Running synthetic, MMM02, run 7
Running synthetic, MMM02, run 8
Running synthetic, MMM02, run 9
Running synthetic, MMM02, run 

,market,mm_type,run,profit
0,synthetic,MMM01,0,60.0
1,synthetic,MMM01,1,405.0
2,synthetic,MMM01,2,353.0
3,synthetic,MMM01,3,0.0
4,synthetic,MMM01,4,419.0
